In [319]:
# import dependencies

import os
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import seaborn as sns
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

import plotly.express as px
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score

## Use the FIPS code for each county to merge datasets

### Create a searchable dict for FIPS code

In [ ]:
# import the FIPS dataset

fips_file = os.path.join('..', '..', 'Data2', 'County_FIPS_Codes.csv')
fips_df = pd.read_csv(fips_file)

# create a searchable dict
fips_dict = {}
for i in range(fips_df.shape[0]):
    key = fips_df.loc[i, 'Name'] + ', ' + fips_df.loc[i, 'State']
    value = fips_df.loc[i, 'FIPS']
    fips_dict.update({key: value})


### Poverty

In [542]:
# poverty dataset

poverty_file = os.path.join('..', '..', 'Data2', 'PovertyEstimates.xls')
poverty_df = pd.read_excel(poverty_file, 
                           header=4, 
#                            index_col='A', 
                           usecols='A,B,C,K,Q,W,Z')

poverty_df = poverty_df[poverty_df['Area_name'].str.find('County')>0] # remove non-county rows
poverty_df.rename({'FIPStxt': 'FIPS',
                   'Stabr': 'State',
                   'Area_name': 'County',
                   'PCTPOVALL_2019': 'poverty_2019_percent_all_age', 
                   'PCTPOV017_2019': 'poverty_2019_percent_age_0-17', 
                   'PCTPOV517_2019': 'poverty_2019_percent_age_5-17', 
                   'MEDHHINC_2019': 'poverty_2019_median_hh_income'}, 
                  axis=1, inplace=True)

poverty_df

,FIPS,State,County,poverty_2019_percent_all_age,poverty_2019_percent_age_0-17,poverty_2019_percent_age_5-17,poverty_2019_median_hh_income
2,1001,AL,Autauga County,12.1,15.9,14.4,58233
3,1003,AL,Baldwin County,10.1,13.5,13.3,59871
4,1005,AL,Barbour County,27.1,41.0,39.5,35972
5,1007,AL,Bibb County,20.3,25.9,25.2,47918
6,1009,AL,Blount County,16.3,21.0,21.4,52902
...,...,...,...,...,...,...,...
3188,56037,WY,Sweetwater County,8.3,10.1,8.2,80639
3189,56039,WY,Teton County,6.0,5.7,4.7,98837
3190,56041,WY,Uinta County,8.5,9.6,7.8,70756
3191,56043,WY,Washakie County,11.1,13.9,12.3,55122


In [560]:
print(poverty_df.shape)
print(poverty_df.isna().sum())
print(poverty_df.dtypes)

(3006, 7)
FIPS                             0
State                            0
County                           0
poverty_2019_percent_all_age     0
poverty_2019_percent_age_0-17    0
poverty_2019_percent_age_5-17    0
poverty_2019_median_hh_income    0
dtype: int64
FIPS                               int64
State                             object
County                            object
poverty_2019_percent_all_age     float64
poverty_2019_percent_age_0-17    float64
poverty_2019_percent_age_5-17    float64
poverty_2019_median_hh_income      int64
dtype: object


### Population density

In [559]:
# population density dataset

density_file = os.path.join('..', '..', 'Data2', 'Average_Household_Size_and_Population_Density_-_County.csv')
density_df = pd.read_csv(density_file)
density_df = density_df[['GEOID',
                        'NAME',
                        'State',
                        'Total_Population',
                        'Population_Density_people_per_square_kilometer']]

density_df.rename({'GEOID': 'FIPS'}, axis=1, inplace=True)
density_df.head()

,FIPS,NAME,State,Total_Population,Population_Density_people_per_square_kilometer
0,1001,Autauga County,Alabama,55200,35.853419
1,1003,Baldwin County,Alabama,208107,50.541504
2,1005,Barbour County,Alabama,25782,11.247981
3,1007,Bibb County,Alabama,22527,13.973114
4,1009,Blount County,Alabama,57645,34.515816


In [558]:
print(density_df.shape)
print(density_df.isna().sum())
print(density_df.dtypes)

(3220, 5)
FIPS                                              0
NAME                                              0
State                                             0
Total_Population                                  0
Population_Density_people_per_square_kilometer    0
dtype: int64
FIPS                                                int64
NAME                                               object
State                                              object
Total_Population                                    int64
Population_Density_people_per_square_kilometer    float64
dtype: object


### Employment and median household income

In [546]:
# employment and median HH income dataset

employment_file = os.path.join('..', '..', 'Data2', 'Unemployment_and_HH_income_median.xlsx')
employment_df = pd.read_excel(employment_file, 
                           header=4, 
#                            index_col='A', 
                           usecols='A:C, CH, CD, BZ, CM'
                             )

employment_df.rename({'FIPS_Code': 'FIPS'}, axis=1, inplace=True)
employment_df.dropna(axis=0, how='any', inplace=True)

employment_df.head()

,FIPS,State,Area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019,Median_Household_Income_2019
0,0,US,United States,4.354932,3.89528,3.669308,65712.0
1,1000,AL,Alabama,4.600000,3.90000,3.000000,51771.0
2,1001,AL,"Autauga County, AL",4.000000,3.60000,2.700000,58233.0
3,1003,AL,"Baldwin County, AL",4.200000,3.60000,2.800000,59871.0
4,1005,AL,"Barbour County, AL",6.000000,5.10000,3.800000,35972.0


In [557]:
print(employment_df.shape)
print(employment_df.isna().sum())
print(employment_df.dtypes)

(3193, 7)
FIPS                            0
State                           0
Area_name                       0
Unemployment_rate_2017          0
Unemployment_rate_2018          0
Unemployment_rate_2019          0
Median_Household_Income_2019    0
dtype: int64
FIPS                              int64
State                            object
Area_name                        object
Unemployment_rate_2017          float64
Unemployment_rate_2018          float64
Unemployment_rate_2019          float64
Median_Household_Income_2019    float64
dtype: object


### Education

In [552]:
# education dataset

education_file = os.path.join('..', '..', 'Data2', 'Education.xls')
education_df = pd.read_excel(education_file, 
                           header=4, 
#                            index_col='A', 
                           usecols='A:C, AR:AU'
                             )
education_df.rename({'FIPS Code': 'FIPS',
                    'Percent of adults with less than a high school diploma, 2015-19':'percent_less_than_high_school',
                    'Percent of adults with a high school diploma only, 2015-19':'percent_high_school',
                    "Percent of adults completing some college or associate's degree, 2015-19":'percent_some_college',
                    "Percent of adults with a bachelor's degree or higher, 2015-19":'percent_bachelor_or_higher'}, 
                     axis=1, inplace=True)
education_df.dropna(axis=0, how='any', inplace=True)

education_df

,FIPS,State,Area name,percent_less_than_high_school,percent_high_school,percent_some_college,percent_bachelor_or_higher
0,0,US,United States,11.998918,26.956844,28.898697,32.145542
1,1000,AL,Alabama,13.819302,30.800268,29.912098,25.468332
2,1001,AL,Autauga County,11.483395,33.588459,28.356571,26.571573
3,1003,AL,Baldwin County,9.193843,27.659616,31.284081,31.862459
4,1005,AL,Barbour County,26.786907,35.604542,26.029837,11.578713
...,...,...,...,...,...,...,...
3278,72145,PR,Vega Baja Municipio,28.428238,26.225822,24.123638,21.222300
3279,72147,PR,Vieques Municipio,28.773281,39.177906,14.049454,17.999357
3280,72149,PR,Villalba Municipio,21.993263,38.366028,19.727892,19.912819
3281,72151,PR,Yabucoa Municipio,29.048897,25.715004,27.233078,18.003019


In [556]:
print(education_df.shape)
print(education_df.isna().sum())
print(education_df.dtypes)

(3273, 7)
FIPS                             0
State                            0
Area name                        0
percent_less_than_high_school    0
percent_high_school              0
percent_some_college             0
percent_bachelor_or_higher       0
dtype: int64
FIPS                               int64
State                             object
Area name                         object
percent_less_than_high_school    float64
percent_high_school              float64
percent_some_college             float64
percent_bachelor_or_higher       float64
dtype: object


### Crime and law enforcement data

#### Dict to convert state names and their abbreviations

In [253]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

### Offense data of 2017

In [565]:
# Offense dataset 

# Import data from .xls file
offense_file = os.path.join('..', '..', 'Data2', 'Table_10_Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2017.xls')
offense_df = pd.read_excel(offense_file, 
                           header=4, 
                           nrows=2517,
                           usecols='A:C, H')
offense_df


# rename columns
offense_df.rename({'Violent\ncrime': 'Violent_crime',
                   'Property\ncrime': 'Property_crime'}, 
                   axis=1,
                   inplace=True)

# change State name to two-letter abbreviation and include to county name
state_list = list(offense_df['State'])

for i in range(len(state_list)):
    
    if type(state_list[i])==str:
        state = state_list[i].split(' - ')[0].title()
    if type(state_list[i])==float:
        state = state.split(' - ')[0]
        
    state_abbrev = us_state_to_abbrev.get(state)
    county = offense_df.loc[i, 'County']
    county_without_number = re.sub(r'[1-9]+', '', county)
    county = re.sub(r' County Police Department', '', county)
    offense_df.loc[i, 'County'] = county + ', ' + state_abbrev
    offense_df.loc[i, 'State'] = state_abbrev

# add FIPS code to offense_df
for i in range(offense_df.shape[0]):
    county = offense_df.loc[i, 'County']
    offense_df.loc[i, 'FIPS'] = fips_dict.get(county)

offense_df.head()

,State,County,Violent_crime,Property_crime,FIPS
0,AL,"Autauga, AL",55.0,384.0,1001.0
1,AL,"Baldwin, AL",163.0,737.0,1003.0
2,AL,"Bibb, AL",3.0,69.0,1007.0
3,AL,"Blount, AL",313.0,812.0,1009.0
4,AL,"Calhoun, AL",18.0,211.0,1015.0


In [567]:
print(offense_df.shape)

(2517, 5)


In [568]:
# divide offense_df into two offense types and remove NaN rows
offense_violent_df = offense_df[['FIPS','State','County','Violent_crime']].dropna(how='any')
offense_property_df = offense_df[['FIPS', 'State','County','Property_crime']].dropna(how='any')

In [569]:
print(offense_violent_df.shape)
print(offense_violent_df.isna().sum())
print(offense_violent_df.dtypes)

(2356, 4)
FIPS             0
State            0
County           0
Violent_crime    0
dtype: int64
FIPS             float64
State             object
County            object
Violent_crime    float64
dtype: object


In [570]:
print(offense_property_df.shape)
print(offense_property_df.isna().sum())
print(offense_property_df.dtypes)

(2426, 4)
FIPS              0
State             0
County            0
Property_crime    0
dtype: int64
FIPS              float64
State              object
County             object
Property_crime    float64
dtype: object


In [506]:
print(offense_violent_df.sort_values(by='Violent_crime', ascending=False).head(20))
print(offense_property_df.sort_values(by='Property_crime', ascending=False).head(20))

         FIPS State            County  Violent_crime
2025  48201.0    TX        Harris, TX         8994.0
142    6037.0    CA   Los Angeles, CA         6118.0
270   12086.0    FL    Miami-Dade, FL         5957.0
974   24005.0    MD     Baltimore, MD         5037.0
273   12095.0    FL        Orange, FL         4367.0
151    6067.0    CA    Sacramento, CA         2612.0
140    6029.0    CA          Kern, CA         2187.0
261   12057.0    FL  Hillsborough, FL         2037.0
972   24003.0    MD  Anne Arundel, MD         2021.0
1842  45079.0    SC      Richland, SC         1977.0
275   12099.0    FL    Palm Beach, FL         1893.0
1834  45045.0    SC    Greenville, SC         1679.0
345   13135.0    GA      Gwinnett, GA         1604.0
324   13063.0    GA       Clayton, GA         1543.0
905   22051.0    LA     Jefferson, LA         1535.0
154    6073.0    CA     San Diego, CA         1486.0
267   12081.0    FL       Manatee, FL         1484.0
1471  36103.0    NY       Suffolk, NY         

### Law enforcement employee of 2017

In [584]:
# import law enforcement employee dataset

# Import data from .xls file
law_emp_file = os.path.join('..', '..', 'Data2', 
    'Table_80_Full_time_Law_Enforcement_Employees_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2017.xls')
law_emp_df = pd.read_excel(law_emp_file, 
                           header=3, 
                           nrows=2451,
#                            usecols='A:C, H'
                          )

# rename columns
law_emp_df.rename({'Total law\nenforcement\nemployees': 'total',
                   'Total\nofficers': 'officers',
                   'Total\ncivilians': 'civilians'
                  }, 
                   axis=1,
                   inplace=True)

# change State name to two-letter abbreviation and include to county name
state_list = list(law_emp_df['State'])

for i in range(len(state_list)):    
    
    if type(state_list[i])==str:
#         print(state_list[i].split('-'))
        state = state_list[i].split('-')[0].title()
    if type(state_list[i])==float:
        state = state.split('-')[0]
        
    state_abbrev = us_state_to_abbrev.get(state)
    county = law_emp_df.loc[i, 'County']
    county_without_number = re.sub(r'[1-9]+', '', county)
    county = re.sub(r' County Police Department', '', county)
    law_emp_df.loc[i, 'County'] = county + ', ' + state_abbrev
    law_emp_df.loc[i, 'State'] = state_abbrev

# add FIPS code to offense_df
for i in range(law_emp_df.shape[0]):
    county = law_emp_df.loc[i, 'County']
    law_emp_df.loc[i, 'FIPS'] = fips_dict.get(county)
    
law_emp_df.head()

,State,County,total,officers,civilians,FIPS
0,AL,"Autauga, AL",69,31,38,1001.0
1,AL,"Baldwin, AL",296,111,185,1003.0
2,AL,"Bibb, AL",10,9,1,1007.0
3,AL,"Calhoun, AL",56,51,5,1015.0
4,AL,"Colbert, AL",53,34,19,1033.0


In [585]:
print(law_emp_df.shape)
print(law_emp_df.isna().sum())
print(law_emp_df.dtypes)

(2451, 6)
State         0
County        0
total         0
officers      0
civilians     0
FIPS         43
dtype: int64
State         object
County        object
total          int64
officers       int64
civilians      int64
FIPS         float64
dtype: object


In [586]:
law_emp_df.sort_values(by='total', ascending=False).head(20)

,State,County,total,officers,civilians,FIPS
130,CA,"Los Angeles, CA",15721,9413,6308,6037.0
466,IL,"Cook, IL",5961,1753,4208,17031.0
241,FL,"Broward, FL",5345,3188,2157,12011.0
2005,TX,"Harris, TX",4609,2250,2359,48201.0
142,CA,"San Diego, CA",4290,2601,1689,6073.0
259,FL,"Miami-Dade, FL",3869,2787,1082,12086.0
136,CA,"Orange, CA",3755,1843,1912,6059.0
138,CA,"Riverside, CA",3643,1831,1812,6065.0
141,CA,"San Bernardino, CA",3509,1957,1552,6071.0
263,FL,"Palm Beach, FL",3476,1577,1899,12099.0


## Merge datasets

### Violent crimes

In [662]:
violent_type_df = offense_violent_df.merge(law_emp_df, how='inner', left_on='FIPS', right_on='FIPS')
violent_type_df = violent_type_df.merge(education_df, how='inner', left_on='FIPS', right_on='FIPS')
violent_type_df = violent_type_df.merge(employment_df, how='inner', left_on='FIPS', right_on='FIPS')
violent_type_df = violent_type_df.merge(density_df, how='inner', left_on='FIPS', right_on='FIPS')
violent_type_df = violent_type_df.merge(poverty_df, how='inner', left_on='FIPS', right_on='FIPS')
violent_type_df.head()

columns_to_keep = [
    'FIPS',
    'Violent_crime',
    'total',
    'officers',
    'civilians',
    'percent_less_than_high_school',
    'percent_high_school',
    'percent_some_college',
    'percent_bachelor_or_higher',
    'Unemployment_rate_2017',
    'Median_Household_Income_2019',
    'Total_Population',
    'Population_Density_people_per_square_kilometer',
    'poverty_2019_percent_all_age',
    'poverty_2019_percent_age_0-17',
    'poverty_2019_percent_age_5-17',
#     'poverty_2019_median_hh_income',
]

rename_dict = {
    'Violent_crime': 'crimes',
    'total': 'total_law_emp',
    'percent_less_than_high_school': 'percent_edu_less_than_high_school',
    'percent_high_school': 'percent_edu_high_school',
    'percent_some_college': 'percent_edu_some_college',
    'percent_bachelor_or_higher': 'percent_edu_bachelor_or_higher',
    'Unemployment_rate_2017': 'unemployment_rate',
    'Median_Household_Income_2019': 'median_HH_Income',
    'Total_Population': 'population',
    'Population_Density_people_per_square_kilometer': 'pop_dens_ppl_per_sqkm',
    'poverty_2019_percent_all_age': 'percent_poverty_all_age',
    'poverty_2019_percent_age_0-17': 'percent_poverty_age_0-17',
    'poverty_2019_percent_age_5-17': 'percent_poverty_age_5-17',
#     'poverty_2019_median_hh_income': 'median_HH_Income_poverty_data',
}

violent_type_df = violent_type_df[columns_to_keep]
violent_type_df.rename(rename_dict, axis=1, inplace=True)
violent_type_df.set_index('FIPS', inplace=True)
violent_type_df.sort_values(by='crimes', ascending=False).head(5)

C:\Users\alexi\.conda\envs\mlenv\lib\site-packages\pandas\core\frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'State_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


,crimes,total_law_emp,officers,civilians,percent_edu_less_than_high_school,percent_edu_high_school,percent_edu_some_college,percent_edu_bachelor_or_higher,unemployment_rate,median_HH_Income,population,pop_dens_ppl_per_sqkm,percent_poverty_all_age,percent_poverty_age_0-17,percent_poverty_age_5-17
FIPS,,,,,,,,,,,,,,,
48201.0,8994.0,4609,2250,2359,18.587011,23.396526,26.566309,31.450153,5.1,61638.0,4602523,1042.130651,15.0,21.3,20.2
6037.0,6118.0,15721,9413,6308,20.866573,20.610870,25.981346,32.541210,4.8,72721.0,10098052,960.742778,13.4,18.3,18.1
12086.0,5957.0,3869,2787,1082,18.551859,27.279696,24.382814,29.785633,4.6,54991.0,2715516,552.187071,15.7,20.8,18.8
24005.0,5037.0,2025,1818,207,8.518860,25.761341,26.362072,39.357727,4.3,76972.0,827625,534.041378,8.9,11.0,11.2
12095.0,4367.0,2124,1482,642,11.466925,24.368099,29.575785,34.589191,3.8,63133.0,1321194,565.200718,12.6,17.9,17.2


In [663]:
print(violent_type_df.shape)
print(violent_type_df.isna().sum())
print(violent_type_df.dtypes)

(2000, 15)
crimes                               0
total_law_emp                        0
officers                             0
civilians                            0
percent_edu_less_than_high_school    0
percent_edu_high_school              0
percent_edu_some_college             0
percent_edu_bachelor_or_higher       0
unemployment_rate                    0
median_HH_Income                     0
population                           0
pop_dens_ppl_per_sqkm                0
percent_poverty_all_age              0
percent_poverty_age_0-17             0
percent_poverty_age_5-17             0
dtype: int64
crimes                               float64
total_law_emp                          int64
officers                               int64
civilians                              int64
percent_edu_less_than_high_school    float64
percent_edu_high_school              float64
percent_edu_some_college             float64
percent_edu_bachelor_or_higher       float64
unemployment_rate              

In [664]:
violent_type_df.to_csv('violent_type_df.csv')

### Property crimes

In [665]:
property_type_df = offense_property_df.merge(law_emp_df, how='inner', left_on='FIPS', right_on='FIPS')
property_type_df = property_type_df.merge(education_df, how='inner', left_on='FIPS', right_on='FIPS')
property_type_df = property_type_df.merge(employment_df, how='inner', left_on='FIPS', right_on='FIPS')
property_type_df = property_type_df.merge(density_df, how='inner', left_on='FIPS', right_on='FIPS')
property_type_df = property_type_df.merge(poverty_df, how='inner', left_on='FIPS', right_on='FIPS')
property_type_df.head()

C:\Users\alexi\.conda\envs\mlenv\lib\site-packages\pandas\core\frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'State_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


,FIPS,State_x,County_x,Property_crime,State_y,County_y,total,officers,civilians,State_x,...,NAME,State_x,Total_Population,Population_Density_people_per_square_kilometer,State_y,County,poverty_2019_percent_all_age,poverty_2019_percent_age_0-17,poverty_2019_percent_age_5-17,poverty_2019_median_hh_income
0,1001.0,AL,"Autauga, AL",384.0,AL,"Autauga, AL",69,31,38,AL,...,Autauga County,Alabama,55200,35.853419,AL,Autauga County,12.1,15.9,14.4,58233
1,1003.0,AL,"Baldwin, AL",737.0,AL,"Baldwin, AL",296,111,185,AL,...,Baldwin County,Alabama,208107,50.541504,AL,Baldwin County,10.1,13.5,13.3,59871
2,1007.0,AL,"Bibb, AL",69.0,AL,"Bibb, AL",10,9,1,AL,...,Bibb County,Alabama,22527,13.973114,AL,Bibb County,20.3,25.9,25.2,47918
3,1015.0,AL,"Calhoun, AL",211.0,AL,"Calhoun, AL",56,51,5,AL,...,Calhoun County,Alabama,115098,73.348688,AL,Calhoun County,17.2,24.6,22.8,47747
4,1033.0,AL,"Colbert, AL",267.0,AL,"Colbert, AL",53,34,19,AL,...,Colbert County,Alabama,54495,35.496950,AL,Colbert County,14.6,19.4,18.6,46683


In [666]:
columns_to_keep = [
    'FIPS',
    'Property_crime',
    'total',
    'officers',
    'civilians',
    'percent_less_than_high_school',
    'percent_high_school',
    'percent_some_college',
    'percent_bachelor_or_higher',
    'Unemployment_rate_2017',
    'Median_Household_Income_2019',
    'Total_Population',
    'Population_Density_people_per_square_kilometer',
    'poverty_2019_percent_all_age',
    'poverty_2019_percent_age_0-17',
    'poverty_2019_percent_age_5-17',
#     'poverty_2019_median_hh_income',
]

rename_dict = {
    'Property_crime': 'crimes',
    'total': 'total_law_emp',
    'percent_less_than_high_school': 'percent_edu_less_than_high_school',
    'percent_high_school': 'percent_edu_high_school',
    'percent_some_college': 'percent_edu_some_college',
    'percent_bachelor_or_higher': 'percent_edu_bachelor_or_higher',
    'Unemployment_rate_2017': 'unemployment_rate',
    'Median_Household_Income_2019': 'median_HH_Income',
    'Total_Population': 'population',
    'Population_Density_people_per_square_kilometer': 'pop_dens_ppl_per_sqkm',
    'poverty_2019_percent_all_age': 'percent_poverty_all_age',
    'poverty_2019_percent_age_0-17': 'percent_poverty_age_0-17',
    'poverty_2019_percent_age_5-17': 'percent_poverty_age_5-17',
#     'poverty_2019_median_hh_income': 'median_HH_Income_poverty_data',
}

property_type_df = property_type_df[columns_to_keep]
property_type_df.rename(rename_dict, axis=1, inplace=True)
property_type_df.set_index('FIPS', inplace=True)
property_type_df.sort_values(by='crimes', ascending=False).head(5)

,crimes,total_law_emp,officers,civilians,percent_edu_less_than_high_school,percent_edu_high_school,percent_edu_some_college,percent_edu_bachelor_or_higher,unemployment_rate,median_HH_Income,population,pop_dens_ppl_per_sqkm,percent_poverty_all_age,percent_poverty_age_0-17,percent_poverty_age_5-17
FIPS,,,,,,,,,,,,,,,
48201.0,42894.0,4609,2250,2359,18.587011,23.396526,26.566309,31.450153,5.1,61638.0,4602523,1042.130651,15.0,21.3,20.2
12086.0,35112.0,3869,2787,1082,18.551859,27.279696,24.382814,29.785633,4.6,54991.0,2715516,552.187071,15.7,20.8,18.8
12095.0,23807.0,2124,1482,642,11.466925,24.368099,29.575785,34.589191,3.8,63133.0,1321194,565.200718,12.6,17.9,17.2
24005.0,23155.0,2025,1818,207,8.518860,25.761341,26.362072,39.357727,4.3,76972.0,827625,534.041378,8.9,11.0,11.2
6037.0,17336.0,15721,9413,6308,20.866573,20.610870,25.981346,32.541210,4.8,72721.0,10098052,960.742778,13.4,18.3,18.1


In [667]:
print(property_type_df.shape)
print(property_type_df.isna().sum())
print(property_type_df.dtypes)

(2071, 15)
crimes                               0
total_law_emp                        0
officers                             0
civilians                            0
percent_edu_less_than_high_school    0
percent_edu_high_school              0
percent_edu_some_college             0
percent_edu_bachelor_or_higher       0
unemployment_rate                    0
median_HH_Income                     0
population                           0
pop_dens_ppl_per_sqkm                0
percent_poverty_all_age              0
percent_poverty_age_0-17             0
percent_poverty_age_5-17             0
dtype: int64
crimes                               float64
total_law_emp                          int64
officers                               int64
civilians                              int64
percent_edu_less_than_high_school    float64
percent_edu_high_school              float64
percent_edu_some_college             float64
percent_edu_bachelor_or_higher       float64
unemployment_rate              

In [670]:
property_type_df.to_csv('property_type_df.csv', index=True)